<a href="https://colab.research.google.com/github/zinob/python_tutorials/blob/main/MonkeyPatch___repr__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `pprint`ing classes that does not have `__repr__` (<main.blah at 0xb1ah>`)

Sometimes you want to use `pprint` or similar to investigate classes that does not have a decent `__repr__` method and thusly just prints  `<blah.blah at 0xb1ah>`. There are a few ways to do this, where the smartest way is probably to update the classes own `__repr__`, if you have access to the source. If that does not work the next best way is probably to write your own function to describe it. But when that, for one reason or another isn't practical you can monkey-patch a generic `__repr__`-function.

In [1]:
class Foo():
  a_class_attribute="I'm invisible"
  def __init__(s,text):
    s.some_attribute=text
    s.lenght=len(text)
  def i_do_nothing(s):
    pass

In [2]:
f = Foo("hello")
g = Foo("world?")
some_dict = {1:f, 2:g} #In this case i was asked about a dict keyed with ints, i isnt common, but that was the question.

# Everything above this line is assumed to be in classes that we cant or wont change.

---



In [3]:
from pprint import pprint

The class `Foo` doesn't have a `__repr__` and thus it is a bit dificult to understand what is really "in it"

In [4]:
pprint(some_dict)

{1: <__main__.Foo object at 0x7f1a9f801150>,
 2: <__main__.Foo object at 0x7f1a9f801110>}


Lets try and create a simple method that works as `__repr__` for "many" objects. 

Sure we could try and be smart, use `inspect.signature` to try and figgure out a propper `__repr__` but at the end of the day it would be a heuristict that *would* fail every now and then, so lets try and be obvious about what we are doing.
The [`object.__dict__`](https://docs.python.org/3/library/stdtypes.html#object.__dict__) attribute contains all writable attributes on an object, that usually translates to all "object variables" that are not inherrited. Lets just create a faux-signature for the class using that, and adding the text `<MONKEY>` to try and make it really obvious that it is not 'the answer'.

This function can be written as a one-liner but I have chosen to make it slightly more verbose for legibility.

In [5]:
def monkeyrepr(self):
  s=[]
  for k,v in self.__dict__.items():
    s.append(f"{k}={repr(v)}")
  class_name = type(self).__name__
  return f'{class_name}(<MONKEY> {", ".join(s)})'

If we want to override `__repr__` for a class we need to get the class-object. The easiest way is often (but not allways) to use the `type()` function. Lets get an object from the dict and get it's type.

In [6]:
target_class=type(some_dict[1])

By over writing the classes `__repr__`-metod with our own, all objects are sudenly able to print a more verbose representation of them selves. But please note one of the caviats with this. The `length` attribute is *not* actually part of the constructors signature, thus if you'd just try to use this to instantiate a new object it would fail.

In [7]:
target_class.__repr__ = monkeyrepr
pprint(some_dict)

{1: Foo(<MONKEY> some_attribute='hello', lenght=5),
 2: Foo(<MONKEY> some_attribute='world?', lenght=6)}


# TL;DR here is the compact version

In [8]:
print(f)
type(f).__repr__ = lambda s: f'{type(s).__name__}(<MONKEY> {", ".join(f"{k}={repr(v)}" for k,v in s.__dict__.items())})'
print(f)

Foo(<MONKEY> some_attribute='hello', lenght=5)
Foo(<MONKEY> some_attribute='hello', lenght=5)
